In [2]:
import os
import os.path as osp
from tcre.env import *
import itertools
%run ../../../src/tcre/logging.py

In [9]:
args = []
for p in itertools.product(
    ['mult_01', 'sngl_01', 'doub_01'], 
    [True, False],
    [True, False],
    ['w2v_frozen', 'w2v_trained', 'denovo'],
    ['S', 'L'],
    [0, .005]
):
    secondary = 'use-secondary' if p[1] else 'no-secondary'
    swaps = 'use-swaps' if p[2] else 'no-swaps'
    args.append(f"--marker-list={p[0]} --{secondary} --{swaps} --wrd-embedding-type={p[3]} --model-size={p[4]} --weight-decay={p[5]}")
args[:5]

['--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=S --weight-decay=0',
 '--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=S --weight-decay=0.005',
 '--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0',
 '--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=L --weight-decay=0.005',
 '--marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_trained --model-size=S --weight-decay=0']

In [12]:
cmd_format = """python /lab/repos/t-cell-relation-extraction/src/tcre/exec/v1/train.py \
--relation-class={relation_class} --device={device} --output-dir={output_dir} \
{args}
"""

output_dir = osp.join(RESULTS_DATA_DIR, 'label-modeling-v3')
for rc in ['inducing_transcription_factor', 'secreted_cytokine', 'inducing_cytokine'][:1]:
    for i, arg in enumerate(args[:1]):
        out_dir = osp.join(output_dir, f'config_{i:04d}')
        cmd = cmd_format.format(relation_class=rc, device='"cuda:1"', output_dir=out_dir, args=arg)
        print('Running command:', cmd)
        !$cmd > $out_dir/logs.txt 2>&1

Running command: python /lab/repos/t-cell-relation-extraction/src/tcre/exec/v1/train.py --relation-class=inducing_transcription_factor --device="cuda:1" --output-dir=/lab/data/results/label-modeling-v3/config_0000 --marker-list=mult_01 --use-secondary --use-swaps --wrd-embedding-type=w2v_frozen --model-size=S --weight-decay=0

INFO:__main__:Found 410 candidates
INFO:__main__:Training config:
{     'device': 'cuda:1',
      'entity_types': ['transcription_factor', 'immune_cell_type'],
      'label': 'relation_class=inducing_transcription_factor:marker_list=mult_01:use_secondary=True:use_swaps=True:use_lower=False:wrd_embedding_type=w2v_frozen:model_size=S:weight_decay=0.0:learning_rate=0.005:vocab_limit=50000:device=cuda:1',
      'learning_rate': 0.005,
      'marker_list': 'mult_01',
      'markers': {     'primary': {'immune_cell_type': ['< @', '@ >'], 'transcription_factor': ['< #', '# >']},
                       'secondary': {'immune_cell_type': ['| @', '@'], 'transcription_factor

In [96]:
!python /lab/repos/t-cell-relation-extraction/src/tcre/exec/v1/train.py \
--relation-class=inducing_transcription_factor \
--marker-list=mult_01 \
--use-secondary \
--use-swaps \
--wrd-embedding-type=w2v_trained \
--model-size=L \
--weight-decay=.005 \
--device='cuda:1' \
--output-dir='/tmp/test'

INFO:__main__:Found 410 candidates
INFO:__main__:Training config:
{     'device': 'cuda:1',
      'entity_types': ['transcription_factor', 'immune_cell_type'],
      'label': 'relation_class=inducing_transcription_factor:marker_list=mult_01:use_secondary=True:use_swaps=True:use_lower=False:wrd_embedding_type=w2v_trained:model_size=L:weight_decay=0.005:learning_rate=0.005:vocab_limit=50000:device=cuda:1',
      'learning_rate': 0.005,
      'marker_list': 'mult_01',
      'markers': {     'primary': {'immune_cell_type': ['< @', '@ >'], 'transcription_factor': ['< #', '# >']},
                       'secondary': {'immune_cell_type': ['| @', '@'], 'transcription_factor': ['| #', '# |']}},
      'model_size': 'L',
      'relation_class': 'inducing_transcription_factor',
      'swaps': {'cytokine': 'CYTOKINE', 'immune_cell_type': 'CELL', 'transcription_factor': 'TF'},
      'use_lower': False,
      'use_secondary': True,
      'use_swaps': True,
      'vocab_limit': 50000,
      'weight_de

In [97]:
!ls /tmp/test

config.json  history.json
